### Seria Lab: add a new ship (part 2)
With the `seria` module being refactored. This _add a new ship_ chapter continues the old part and demonstrates how to add a new ship from a design file and add it to the player's profile with updated API.

In [51]:
import seria

profile_path = 'profile.seria'
profile = seria.load(profile_path)
rook = seria.load('sample/Rook.seria')

In [ ]:
# verify that the dump is identical to the original file
seria.dump(profile, 'profile.seria')

source = open('sample/profile.seria', encoding='cp1251')
target = open('profile.seria', encoding='cp1251')

print(source.read() == target.read())

source.close()
target.close()

In [52]:
# get existing unique id values

unique_ids = set()

file = open(profile_path, 'r', encoding='cp1251')
lines = file.readlines()
for line in lines:
    name, value = seria._match_attribute(line)
    if name == 'm_id':
        unique_ids.add(int(value))

In [53]:
# make new id unique from existing ids
next_id = max(unique_ids) + 1

# obtain correct attribute values from profile
next_creature_id = profile.get_attribute('nextCreatureId')

p_fleet = profile.get_node_if(
    lambda node: node.get_attribute('m_name') == 'MARK')

alignment = p_fleet.get_attribute('m_alignment')
escadra_id = p_fleet.get_attribute('m_id')

p_ships = p_fleet.filter_nodes(lambda node: node.header == 'm_children=7')

# make new escadra index for the new ship
escadra_indexes = set()
for ship in p_ships:
    # Frame > Body > Creature
    creature = ship.get_node_by_class('Frame').get_node(0).get_node(0)
    escadra_indexes.add(int(creature.get_attribute('m_escadra_index')))

### Steps to configure a ship
#### root _Node_ node
1. Add a `header` of `m_children=7` to the beginning of the node.
2. Make a new unique `m_id` for the root _Node_ node and update other nodes that reference the old value.
3. Set `m_state` to `2`; this follows the values of other ships, which are `2`.
4. Set `m_master_id` to be the `m_id` of the escadra which this ship belongs to.
#### _Body_ nodes of root _Node_ node
5. For all depth 2 _Body_ node of the root Node node. Make a new unique `m_id` for each _Body_ node. Update other nodes that reference the old value, this includes nodes in the same depth and nodes in a deeper depth (child nodes).
6. Make a new unique `m_id` for the _Frame_ node and update other nodes that reference the old value.
#### _Frame_ node
7. For all depth 3 _Body_ node which is the child of Frame node. Make a new unique `m_id` for these nodes and update all other nodes that reference the old value.
#### _Creature_ node
8. For _Creature_ node, make a new unique `m_id` for it and update other nodes that reference the old value. _Creature_'s `m_owner_id` should reference its own `m_id`.
9. set `creatureId` to the `nextCreatureId` obtained from the root node of the profile. Also update `nextCreatureId` because this one has been used by our new ship.
10. set `m_escadra.id` to be the `m_id` of the escadra which this ship belongs to.
11. set `m_alignment` to be `1` (friendly) or `-1` (enemy). This can be sampled from other ships in the same fleet.
12. set `m_tarkhan` to be `DAUD`, or other NPCs that has joined your team.
13. set `m_radiation_extra` to be `1` or can be sampled from other ships (but `1` works for me, so I just keep it).
14. set `m_tele_crew_total` to be the same value as `m_tele_crew_capacity` obtained from the _Creature_ node. Same value means the crew is full.
15. set `m_moral` to be `10`. You don't wanna ship to be added in a low moral, right?
16. set `m_rad_seekness_timer` to be `5`, or you can sample and average it from other ships.
17. set `shipLoadTime` to be `0`.
18. set `wasAddedToPlayerEscadra` to be `true`.
#### Bugs relate to misconfiguring attributes:
If not set `m_rad_seekness_timer` and `m_radiation_extra`, in the game, when hovering the cursor on the ship select button and move away. The fuel will drop about 0.5% each time we repeat the action.  
The order of attributes does matter. It is clear that if not set the attribute in the correct order. For example, set `m_tele_crew_total` after `m_tele_crew_capacity`, the `m_tele_crew_total` will be discarded by the game. This missing attribute can cause further bugs in the game.  
The new `seria` module is designed to maintain the order of attributes and has provided APIs that allow you to insert attributes before and after another attribute. See the code sample below.

In [54]:
# update ship design with needed attributes obtained from profile
rook.header = 'm_children=7'
rook.put_attribute_after('m_state', '2', 'm_name')
rook.put_attribute_after('m_master_id', escadra_id, 'm_state')

# m_id for root Node node of the ship
rook.update_attribute('m_id', str(next_id))

# update all depth 2 nodes with new m_id
for node in rook.get_nodes():
    old_id = node.get_attribute('m_id')

    next_id += 1
    node.update_attribute('m_id', str(next_id))

    # update all joint nodes with new m_id reference
    for node in rook.get_nodes():
        node.update_attribute_by_value(old_id, str(next_id))

rook_frame = rook.get_node_by_class('Frame')
next_id += 1
rook_frame.update_attribute('m_id', str(next_id))

# update all depth 3 nodes in frame node with new m_id
for node in rook_frame.get_nodes():
    old_id = node.get_attribute('m_id')

    next_id += 1
    node.update_attribute('m_id', str(next_id))

    # update all joint nodes with new m_id reference
    for node in rook.get_nodes():
        node.update_attribute_by_value(old_id, str(next_id))

rook_creature = rook_frame.get_node(0).get_node(0)
next_id += 1
old_creature_id = rook_creature.get_attribute('m_id')
rook_creature.update_attribute('m_id', str(next_id))
rook_creature.set_attribute('m_owner_id', str(next_id))
rook.update_attribute_by_value(old_creature_id, str(next_id))

rook_creature.set_attribute('creatureId', next_creature_id)
profile.set_attribute('nextCreatureId', str(int(next_creature_id) + 1))

rook_creature.put_attribute_after('m_escadra.id', escadra_id, 'm_health_lock')
rook_creature.put_attribute_after('m_escadra_index', str(
    max(escadra_indexes) + 1), 'm_escadra.id')
rook_creature.put_attribute_after('m_alignment', alignment, 'm_playable')
rook_creature.put_attribute_after('m_tarkhan', 'DAUD', 'm_alignment')
rook_creature.put_attribute_after('m_radiation_extra', '1', 'm_tarkhan')

crew_capacity = rook_creature.get_attribute('m_tele_crew_capacity')
rook_creature.put_attribute_before(
    'm_tele_crew_total', crew_capacity, 'm_tele_crew_capacity')
rook_creature.put_attribute_before('m_moral', '10', 'creatureId')
rook_creature.put_attribute_before('m_rad_seekness_timer', '5', 'shipLoadTime')
rook_creature.put_attribute_before('shipLoadTime', '0', 'm_tele_fuel_on')

rook_creature.set_attribute('wasAddedToPlayerEscadra', 'true')

In [55]:
p_fleet.put_node_after(rook, p_ships[-1])

seria.dump(profile, 'profile.seria')

# seria.dump(rook, 'Rook.seria')

# rook_sample = p_fleet.get_node_if(
#     lambda node: node.get_attribute('m_name') == 'Rook')
# seria.dump(rook_sample, 'Rook_sample.seria')